## INTEL IMAGE CLASSIFICATION

In [ ]:
#connecting google drive to colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#IMPORTING FILE FROM KAGGLE
#Go to Kaggle profice .... Scroll down ... There's API.....You'll get username and password from there
import os
os.environ['KAGGLE_USERNAME'] = "nikki2501" # username from the json file
os.environ['KAGGLE_KEY'] = "18a26daea5b258038f4b2a03cdfee09f" # key from the json file

In [ ]:
#downloading Kaggle dataset
!kaggle datasets download -d puneet6060/intel-image-classification

100% 345M/346M [00:02<00:00, 126MB/s]
100% 346M/346M [00:02<00:00, 123MB/s]


In [ ]:
# Unzip training data
from zipfile import ZipFile
file_name = "/content/intel-image-classification.zip"
with ZipFile(file_name, 'r') as zip:
  zip.extractall()
  print('done')

done


In [ ]:
pip install split-folders
#this library is used to divide data to train test n val 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#IMPORTING necessary Libraries
# from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.models import Sequential
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint

import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Activation, Dense, Flatten, BatchNormalization, Conv2D, MaxPool2D,Dropout,GlobalAveragePooling2D
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import itertools
import os
import shutil
import random
import glob
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.image import imread
import pathlib
from sklearn.metrics import classification_report, confusion_matrix

# %matplotlib inline

In [ ]:
import splitfolders
# Split with a ratio.
# To only split into training and validation set, set a tuple to `ratio`, i.e, `(.8, .2)`.
splitfolders.ratio("/content/seg_train/seg_train", output="FINAL",
    # seed=1337, 
    ratio=(.8, .2), group_prefix=None, move=False) # default values

Copying files: 14034 files [00:02, 5692.01 files/s]


In [ ]:
#Mentioning path to all directories
train_dir ='/content/FINAL/train'
val_dir = '/content/FINAL/val'
test_dir ='/content/seg_test/seg_test'

In [ ]:
#Checking the class labels of train directory
labels = os.listdir(train_dir)
print(labels)

['mountain', 'street', 'forest', 'glacier', 'buildings', 'sea']


In [ ]:
#defining image size and batch size (batch size is Number of samples per batch of computation )
img_width=331
img_height=331
batch_size=128

In [ ]:
#ImageDataGenerator is used Generate batches of tensor image data with real-time data augmentation.
train_datagen = ImageDataGenerator(rescale = 1./255)
                                  #  rotation_range=30,rescale = 1/255,
                                  #  zoom_range=0.4,
                                  #  horizontal_flip=True)
#all the above commented can be when we have to augment the images
#flow_from_directory Takes the path to a directory, and generates batches of augmented/normalized data
#to understand more about flow_from_directory refer this: https://studymachinelearning.com/keras-imagedatagenerator-with-flow_from_directory/
#class_mode : One of "categorical", "binary", "sparse", "input", or None. Default: "categorical".
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=batch_size,
                                                    class_mode='categorical',
                                                    target_size=(img_height, img_width))

Found 11224 images belonging to 6 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)#rescale = 1/255

test_generator = test_datagen.flow_from_directory(test_dir,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  target_size=(img_height, img_width))

Found 3000 images belonging to 6 classes.


In [ ]:
val_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = val_datagen.flow_from_directory(val_dir,
                                                 batch_size=batch_size,
                                                 class_mode='categorical',
                                                 target_size=(img_height, img_width))

Found 2810 images belonging to 6 classes.


In [ ]:
inference_dir= "/content/drive/MyDrive/Inference/Inference Data(DX)"

In [ ]:
inf_datagen = ImageDataGenerator(rescale = 1./255)#rescale = 1/255

inf_generator = inf_datagen.flow_from_directory(inference_dir,
                                                  batch_size=batch_size,
                                                  class_mode='categorical',
                                                  target_size=(img_height, img_width))

Found 3004 images belonging to 6 classes.


In [ ]:
# def scale_image(img,label):
#   img = tf.cast(img,tf.float32)
#   img = img/255.0
#   img = tf.image.resize(img, (331,331))
#   return img,label 

FROM here ownwards we start to use Keras models
to know in deep about the models refer this : https://keras.io/api/applications/

## **NasNetLarge**

In [ ]:
from keras.applications.nasnet import NASNetLarge

In [ ]:
input_shape=(331,331,3)

Transfer learning is a machine learning method where a model developed for a task is reused as the starting point for a model on a second task. using transfer learning.... imports model from it's stored location. no need to write each and every layer

include_top false means because the fully connected layers at the end can only take fixed size inputs, which has been previously defined by the input shape and all processing in the convolutional layers. Any change to the input shape will change the shape of the input to the fully connected layers, making the weights incompatible (matrix sizes don't match and cannot be applied).

Imagenet weights is used for images classification,transfer learning model models use the imagenet weights

In [ ]:
#transfer learning used , explained above
base = NASNetLarge(input_shape,
                        include_top=False,
                        weights='imagenet')

base.trainable = False

343610240/343610240 [==============================] - 3s 0us/step


In [ ]:
#Adding layers to our model, pooling it
#Then mentioning number of classes we have(6 in this dataset)

inputs = keras.Input(shape=(331,331,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
#Compiling the model 
#optimizer is used to reduce the losses
#different type of metric explained here :"https://keras.io/api/metrics/"
#Losses : The purpose of loss functions is to compute the quantity that a model should seek to minimize during training.
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
#fitting the model on daatset and traing our machine, Train_gen and val_gen gives the accuracy by which it works and epochs means how many number of time it gonna run to model again and again
model.fit(train_generator, epochs=10, validation_data=val_generator)

In [ ]:
#Saving model in .h5 file 
model.save('/content/drive/MyDrive/Colab Notebooks/NasNetLarge.h5')

In [ ]:
#checking the accuracy of test dataset
model.evaluate(test_generator)

24/24 [==============================] - 105s 4s/step - loss: 1.7335 - categorical_accuracy: 0.2197


[1.7335163354873657, 0.21966665983200073]

In [ ]:
new_model = keras.models.load_model('/content/drive/MyDrive/NasNetLarge.h5')

In [ ]:
#checking the accuracy of inference dataset
new_model.evaluate(inf_generator)

24/24 [==============================] - 886s 37s/step - loss: 1.0159 - categorical_accuracy: 0.7713


[1.0159363746643066, 0.7713049054145813]

### Same model with 30epochs

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(train_generator, epochs=30, validation_data=val_generator)

Epoch 1/30
88/88 [==============================] - 448s 5s/step - loss: 0.4235 - categorical_accuracy: 0.8579 - val_loss: 0.2851 - val_categorical_accuracy: 0.9007
Epoch 2/30
88/88 [==============================] - 420s 5s/step - loss: 0.2360 - categorical_accuracy: 0.9138 - val_loss: 0.2630 - val_categorical_accuracy: 0.9096
Epoch 3/30
88/88 [==============================] - 421s 5s/step - loss: 0.2081 - categorical_accuracy: 0.9228 - val_loss: 0.2509 - val_categorical_accuracy: 0.9117
Epoch 4/30
88/88 [==============================] - 420s 5s/step - loss: 0.1914 - categorical_accuracy: 0.9300 - val_loss: 0.2377 - val_categorical_accuracy: 0.9192
Epoch 5/30
88/88 [==============================] - 421s 5s/step - loss: 0.1789 - categorical_accuracy: 0.9342 - val_loss: 0.2346 - val_categorical_accuracy: 0.9142
Epoch 6/30
88/88 [==============================] - 421s 5s/step - loss: 0.1673 - categorical_accuracy: 0.9383 - val_loss: 0.2337 - val_categorical_accuracy: 0.9189
Epoch 7/30

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/NasNetLarge_30epochs.h5')

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 93s 4s/step - loss: 0.2202 - categorical_accuracy: 0.9203


[0.22016750276088715, 0.9203333258628845]

In [ ]:
model.evaluate(inf_generator)

24/24 [==============================] - 133s 5s/step - loss: 1.3642 - categorical_accuracy: 0.7630


[1.3641738891601562, 0.7629826664924622]

## Trying different models , checking the accuracy and saving :

#**NasNetMobile**

In [ ]:
from keras.applications.nasnet import NASNetMobile

In [ ]:
input_shape=(224,224,3)

In [ ]:
base = NASNetMobile(    input_shape=None,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
)
base.trainable = False

24227760/24227760 [==============================] - 1s 0us/step


In [ ]:
inputs = keras.Input(shape=(224,224,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, epochs=10, validation_data=val_generator)

# InceptionV3

In [ ]:
from keras.applications import InceptionV3

In [ ]:
base=InceptionV3 (
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

In [ ]:
base.trainable = False
inputs = keras.Input(shape=(299,299,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10


In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(train_generator, epochs=100, validation_data=val_generator)

Epoch 1/100
88/88 [==============================] - 87s 876ms/step - loss: 1.7676 - categorical_accuracy: 0.4735 - val_loss: 1.7359 - val_categorical_accuracy: 0.6843
Epoch 2/100
88/88 [==============================] - 65s 739ms/step - loss: 1.7069 - categorical_accuracy: 0.7177 - val_loss: 1.6771 - val_categorical_accuracy: 0.7370
Epoch 3/100
88/88 [==============================] - 65s 738ms/step - loss: 1.6491 - categorical_accuracy: 0.7511 - val_loss: 1.6208 - val_categorical_accuracy: 0.7495
Epoch 4/100
88/88 [==============================] - 65s 737ms/step - loss: 1.5937 - categorical_accuracy: 0.7564 - val_loss: 1.5669 - val_categorical_accuracy: 0.7616
Epoch 5/100
88/88 [==============================] - 65s 739ms/step - loss: 1.5405 - categorical_accuracy: 0.7692 - val_loss: 1.5154 - val_categorical_accuracy: 0.7751
Epoch 6/100
88/88 [==============================] - 65s 738ms/step - loss: 1.4899 - categorical_accuracy: 0.7831 - val_loss: 1.4661 - val_categorical_accuracy:

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 17s 714ms/step - loss: 0.4687 - categorical_accuracy: 0.8307


[0.46870434284210205, 0.8306666612625122]

## **ResNet152V2**

In [ ]:
from keras.applications.resnet_v2 import ResNet152V2

In [ ]:
input_shape=(224,224,3)

In [ ]:
base=ResNet152V2( input_shape,
    # include_top=True,
    weights='imagenet',
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation='softmax'
)

base.trainable = False


In [ ]:
inputs = keras.Input(shape=(224,224,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])
model.fit(train_generator, epochs=10, validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 236s 3s/step - loss: 1.7619 - categorical_accuracy: 0.5426 - val_loss: 1.7265 - val_categorical_accuracy: 0.7242
Epoch 2/10
88/88 [==============================] - 225s 3s/step - loss: 1.6937 - categorical_accuracy: 0.7414 - val_loss: 1.6603 - val_categorical_accuracy: 0.7569
Epoch 3/10
88/88 [==============================] - 225s 3s/step - loss: 1.6290 - categorical_accuracy: 0.7558 - val_loss: 1.5973 - val_categorical_accuracy: 0.7701
Epoch 4/10
88/88 [==============================] - 225s 3s/step - loss: 1.5676 - categorical_accuracy: 0.7684 - val_loss: 1.5374 - val_categorical_accuracy: 0.7754
Epoch 5/10
88/88 [==============================] - 225s 3s/step - loss: 1.5090 - categorical_accuracy: 0.7803 - val_loss: 1.4807 - val_categorical_accuracy: 0.7833
Epoch 6/10
88/88 [==============================] - 224s 3s/step - loss: 1.4536 - categorical_accuracy: 0.7839 - val_loss: 1.4269 - val_categorical_accuracy: 0.7836
Epoch 7/10

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 49s 2s/step - loss: 1.2425 - categorical_accuracy: 0.7870


[1.2425174713134766, 0.7870000004768372]

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/resnet152v2')

## **ResNet101**

In [ ]:
from keras.applications import ResNet101

In [ ]:
base=ResNet101(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    # **kwargs
)

179648224/179648224 [==============================] - 4s 0us/step


In [ ]:
base.trainable = False

In [ ]:
inputs = keras.Input(shape=(224,224,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 189s 2s/step - loss: 1.7899 - categorical_accuracy: 0.2094 - val_loss: 1.7882 - val_categorical_accuracy: 0.1790
Epoch 2/10
88/88 [==============================] - 169s 2s/step - loss: 1.7870 - categorical_accuracy: 0.1826 - val_loss: 1.7855 - val_categorical_accuracy: 0.1804
Epoch 3/10
88/88 [==============================] - 169s 2s/step - loss: 1.7844 - categorical_accuracy: 0.1834 - val_loss: 1.7830 - val_categorical_accuracy: 0.1872
Epoch 4/10
88/88 [==============================] - 168s 2s/step - loss: 1.7819 - categorical_accuracy: 0.1916 - val_loss: 1.7805 - val_categorical_accuracy: 0.2021
Epoch 5/10
88/88 [==============================] - 169s 2s/step - loss: 1.7794 - categorical_accuracy: 0.2166 - val_loss: 1.7781 - val_categorical_accuracy: 0.2117
Epoch 6/10
88/88 [==============================] - 169s 2s/step - loss: 1.7769 - categorical_accuracy: 0.2050 - val_loss: 1.7756 - val_categorical_accuracy: 0.2295
Epoch 7/10

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 39s 2s/step - loss: 1.7649 - categorical_accuracy: 0.2423


[1.7649445533752441, 0.2423333376646042]

In [ ]:
   model.save('/content/drive/MyDrive/Colab Notebooks/resnet152v2')

## **DenseNet169**

In [ ]:
from keras.applications import DenseNet169

In [ ]:
base=DenseNet169(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
)

In [ ]:
base.trainable = False

In [ ]:
inputs = keras.Input(shape=(224,224,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 142s 1s/step - loss: 1.7545 - categorical_accuracy: 0.5624 - val_loss: 1.7183 - val_categorical_accuracy: 0.6488
Epoch 2/10
88/88 [==============================] - 106s 1s/step - loss: 1.6868 - categorical_accuracy: 0.6709 - val_loss: 1.6524 - val_categorical_accuracy: 0.6694
Epoch 3/10
88/88 [==============================] - 106s 1s/step - loss: 1.6233 - categorical_accuracy: 0.6902 - val_loss: 1.5902 - val_categorical_accuracy: 0.6964
Epoch 4/10
88/88 [==============================] - 110s 1s/step - loss: 1.5629 - categorical_accuracy: 0.7210 - val_loss: 1.5314 - val_categorical_accuracy: 0.7185
Epoch 5/10
88/88 [==============================] - 108s 1s/step - loss: 1.5059 - categorical_accuracy: 0.7260 - val_loss: 1.4757 - val_categorical_accuracy: 0.7231
Epoch 6/10
88/88 [==============================] - 106s 1s/step - loss: 1.4521 - categorical_accuracy: 0.7296 - val_loss: 1.4235 - val_categorical_accuracy: 0.7324
Epoch 7/10

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 29s 1s/step - loss: 1.2482 - categorical_accuracy: 0.7573


[1.2482304573059082, 0.7573333382606506]

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/DenseNet169')

## **EfficientNetV2M**

In [ ]:
from keras.applications import EfficientNetV2M

In [ ]:
base= EfficientNetV2M(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
    include_preprocessing=True,
)


base.trainable = False

In [ ]:
inputs = keras.Input(shape=(480,480,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 230s 2s/step - loss: 1.7911 - categorical_accuracy: 0.1758 - val_loss: 1.7904 - val_categorical_accuracy: 0.1790
Epoch 2/10
88/88 [==============================] - 200s 2s/step - loss: 1.7902 - categorical_accuracy: 0.1790 - val_loss: 1.7897 - val_categorical_accuracy: 0.1790
Epoch 3/10
88/88 [==============================] - 200s 2s/step - loss: 1.7896 - categorical_accuracy: 0.1790 - val_loss: 1.7892 - val_categorical_accuracy: 0.1790
Epoch 4/10
88/88 [==============================] - 201s 2s/step - loss: 1.7890 - categorical_accuracy: 0.1790 - val_loss: 1.7886 - val_categorical_accuracy: 0.1790
Epoch 5/10
88/88 [==============================] - 201s 2s/step - loss: 1.7885 - categorical_accuracy: 0.1790 - val_loss: 1.7881 - val_categorical_accuracy: 0.1790
Epoch 6/10
88/88 [==============================] - 201s 2s/step - loss: 1.7879 - categorical_accuracy: 0.1790 - val_loss: 1.7876 - val_categorical_accuracy: 0.1790
Epoch 7/10

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 46s 2s/step - loss: 1.7848 - categorical_accuracy: 0.1750


[1.784754991531372, 0.17499999701976776]

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/EfficientNetV2M')

## **xception**

In [ ]:
from keras.applications import Xception

In [ ]:
base= Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
)

base.trainable = False

In [ ]:
inputs = keras.Input(shape=(299,299,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 161s 2s/step - loss: 1.7591 - categorical_accuracy: 0.5790 - val_loss: 1.7258 - val_categorical_accuracy: 0.7306
Epoch 2/10
88/88 [==============================] - 146s 2s/step - loss: 1.6955 - categorical_accuracy: 0.7431 - val_loss: 1.6635 - val_categorical_accuracy: 0.7847
Epoch 3/10
88/88 [==============================] - 147s 2s/step - loss: 1.6349 - categorical_accuracy: 0.7830 - val_loss: 1.6040 - val_categorical_accuracy: 0.7968
Epoch 4/10
88/88 [==============================] - 147s 2s/step - loss: 1.5770 - categorical_accuracy: 0.7878 - val_loss: 1.5473 - val_categorical_accuracy: 0.8014
Epoch 5/10
88/88 [==============================] - 147s 2s/step - loss: 1.5217 - categorical_accuracy: 0.7934 - val_loss: 1.4933 - val_categorical_accuracy: 0.8068
Epoch 6/10
88/88 [==============================] - 147s 2s/step - loss: 1.4693 - categorical_accuracy: 0.7985 - val_loss: 1.4418 - val_categorical_accuracy: 0.8043
Epoch 7/10

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 31s 1s/step - loss: 1.2611 - categorical_accuracy: 0.7890


[1.261128306388855, 0.7889999747276306]

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/xception_1')

## **MobileNet**

In [ ]:
from keras.applications import MobileNet

In [ ]:
base=MobileNet(
    # input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
    # **kwargs
)
base.trainable= False

17225924/17225924 [==============================] - 0s 0us/step


In [ ]:
inputs = keras.Input(shape=(224,224,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 50s 529ms/step - loss: 1.7543 - categorical_accuracy: 0.5375 - val_loss: 1.7151 - val_categorical_accuracy: 0.7128
Epoch 2/10
88/88 [==============================] - 38s 435ms/step - loss: 1.6809 - categorical_accuracy: 0.7270 - val_loss: 1.6438 - val_categorical_accuracy: 0.7552
Epoch 3/10
88/88 [==============================] - 39s 444ms/step - loss: 1.6119 - categorical_accuracy: 0.7576 - val_loss: 1.5766 - val_categorical_accuracy: 0.7797
Epoch 4/10
88/88 [==============================] - 38s 434ms/step - loss: 1.5468 - categorical_accuracy: 0.7675 - val_loss: 1.5131 - val_categorical_accuracy: 0.7904
Epoch 5/10
88/88 [==============================] - 38s 433ms/step - loss: 1.4854 - categorical_accuracy: 0.7775 - val_loss: 1.4533 - val_categorical_accuracy: 0.7947
Epoch 6/10
88/88 [==============================] - 39s 439ms/step - loss: 1.4273 - categorical_accuracy: 0.7808 - val_loss: 1.3967 - val_categorical_accuracy: 0.782

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 10s 393ms/step - loss: 1.2090 - categorical_accuracy: 0.7690


[1.209046483039856, 0.7689999938011169]

In [ ]:
model.save('/content/drive/MyDrive/Colab Notebooks/MobileNet')

#**EfficientNetB2**

In [ ]:
from keras.applications import EfficientNetB2

In [ ]:
base= EfficientNetB2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
    # **kwargs
)

base.trainable=False

37432240/37432240 [==============================] - 0s 0us/step


In [ ]:
inputs = keras.Input(shape=(260,260,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

Epoch 1/10
88/88 [==============================] - 98s 1s/step - loss: 1.7911 - categorical_accuracy: 0.1757 - val_loss: 1.7904 - val_categorical_accuracy: 0.1790
Epoch 2/10
88/88 [==============================] - 81s 915ms/step - loss: 1.7903 - categorical_accuracy: 0.1790 - val_loss: 1.7898 - val_categorical_accuracy: 0.1790
Epoch 3/10
88/88 [==============================] - 81s 918ms/step - loss: 1.7897 - categorical_accuracy: 0.1790 - val_loss: 1.7893 - val_categorical_accuracy: 0.1790
Epoch 4/10
88/88 [==============================] - 81s 914ms/step - loss: 1.7893 - categorical_accuracy: 0.1790 - val_loss: 1.7889 - val_categorical_accuracy: 0.1790
Epoch 5/10
88/88 [==============================] - 81s 914ms/step - loss: 1.7887 - categorical_accuracy: 0.1790 - val_loss: 1.7884 - val_categorical_accuracy: 0.1790
Epoch 6/10
88/88 [==============================] - 81s 914ms/step - loss: 1.7883 - categorical_accuracy: 0.1793 - val_loss: 1.7879 - val_categorical_accuracy: 0.1794
E

In [ ]:
model.evaluate(test_generator)

24/24 [==============================] - 19s 785ms/step - loss: 1.7855 - categorical_accuracy: 0.1843


[1.7854835987091064, 0.18433333933353424]

# **EfficientNetB3**

In [ ]:
from keras.applications import EfficientNetB3

In [ ]:
base =EfficientNetB3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
    # **kwargs
)

In [ ]:
inputs = keras.Input(shape=(300,300,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

In [ ]:
model.evaluate(test_generator)

## **EfficientNetB4**

In [ ]:
from keras.applications import EfficientNetB4

In [ ]:
base=EfficientNetB4(
    # include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
    # **kwargs
)

base.trainable-False

78864416/78864416 [==============================] - 2s 0us/step


1

In [ ]:
inputs = keras.Input(shape=(380,380,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)

In [ ]:
model.evaluate(test_generator)

In [ ]:
path = '/content/drive/MyDrive/Colab Notebooks/all'
check_point = ModelCheckpoint( filepath = path,
                                          monitor = " val_acc " ,
                                          mode = " max " ,
                                          save_best_only = True ,
                                          save_weights_only= True
                                          )

## **InceptionResNetV2**

In [ ]:
from keras.applications import InceptionResNetV2

In [ ]:
base=InceptionResNetV2(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    # classes=1000,
    classifier_activation="softmax",
    # **kwargs
)

base.trainable=False

In [ ]:
inputs = keras.Input(shape=(380,380,3))
# We make sure that the base_model is running in inference mode here,
# by passing `training=False`. This is important for fine-tuning, as you will
# learn in a few paragraphs.
x = base(inputs, training=False)
# Convert features of shape `base_model.output_shape[1:]` to vectors
# x = keras.layers.GlobalAveragePooling2D()(x)
# A Dense classifier with a single unit (binary classification)
outputs = keras.layers.Dense(6)(x)
model = keras.Model(inputs, outputs)

In [ ]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=False),
              metrics=[keras.metrics.CategoricalAccuracy()])

In [ ]:
model.fit(train_generator, 
          epochs=10, 
          validation_data=val_generator)